## Embeddings
这里不但包括了单词的embeddings，同时还包括了各种其他的属性的embedding和position的。<br>
不同feature之间存在三种融合的方式，即concat，sum，mlp。<br>
模型是由多个nn.Embedding模型(针对不同的features)组合而成，根据输入和参数的不同，呈现不同数目及大小的模型。<br>
输出的embedding已经融合了所要求的features
#### 属性
- self.positional_encoding: Boolean, action = 'store_true'是否使用positional_encoding.
- self.pe: [max_len, 1, word_vec_len] max_len是最大可能的位置数量。保存的代表不同位置的向量。使向量长度为词向量的长度，是为了方便和单词的词向量进行结合。
- self.feat_merge: choices=['concat', 'sum', 'mlp']。指定不同features embedding之间的融合方式
- feat_exp = opt.feat_vec_exponent: 指定如果merge方式使用concat的话，融合后的向量长度，应该为对应feature的值域空间的feat_exp次方。default = 0.7
- vocab_sizes：列表，指各种属性的值域空间的大小。
- emb_sizes: 列表，指各种属性生成的向量的长度大小。
- self.mlp: 当使用mlp融合时，mlp的结构
- self.emb_luts: nn.ModuleList保存各个feature的embedding模型。

#### 方法
- __init__(self, opt, dicts, feature_dicts = None): 设置dict，feature dict的embedding模型。
- word_lut(self): 输出单词的embedding模型
- embedding_size(self): 返回词向量的长度
- make_positional_encoding(self, dim, mac_len): 生成positional embedding
- load_pretrained_vectors(self, emb_file): 顾名思义
- merge(self, features): 融合
- forward(self, src_input_): 运行方法

## Encoder
设定MT系统的Encoder部分，可以调节层的数量及每个层中节点的数量以及节点的类型。模型输入由前置的embedding模型得到。<br>
目前支持三种类型的层：即GRU，LSTM和Transformer。<br>
#### 属性
- self.layers: int, encoder层的个数
- self.num_directions: int [0, 1] 是否bidirection
- self.hidden_size: 每个隐藏层的节点的数量，这里指的是每个方向的数量，如果是bidirection，那么总的数量应该是它的两倍。
- self.embeddings：embedding模型
- self.encoder_layer: choices = ['GRU', 'LSTM', 'Transformer'] 指定每个层的用的结构类型
- self.transformer: 如果encoder_layer是Transformer，设定为对应的transformer模型（onmt.modules.TransformerEncoder）列表（每一项代表一层）
- self.rnn: 如果encoder_layer不是Transformer，设定为对应的rnn模型（nn.LSTM/nn.GRU）列表

#### assert
assert opt.rnn_size % self.num_direction == 0

#### 方法
__init__(self, opt, dicts, feature_dicts = None)
forward(self, intput, lengths = None, hidden = None): length是指batch_size，hidden是隐藏层的初始值。

#### 图
还木画

## Decoder
设定MT模型的Decoder部分，可以设定层的数量及每个层中节点的数量以及节点的类型，attention的类型。<br>
目前支持的层的类型有：GRU，LSTM，Transformer<br>
目前支持的attention的类型有：dot, general, mlp, copy<br>
目前支持的结合context的方式有：source, target, both<br>

#### 属性
- self.layers: int, decoder层的个数，encoder和decoder层数相同
- self.decoder_layer: int, decoder每个层的类型
- self._coverage: action='store_true' 指定attention是否要使用coverage。用额外的向量来记录已经翻译到什么地方，从而实现coverage。
- self.hidden_size: 因为不用考虑bidrectional所以等于opt.rnn_size
- self.input_feed: 指定是否使用额外的context信息作为输入信息，如果是将扩充input_size(原始为word_vec_size)，default = 1
- self.transformer: 如果decoder_layer是Transformer，设定为对应的Transformer模型（onmt.modules.TransformerDecoder()）列表
- self.rnn: 如果decoder_layer不是Transformer，设定为对应的rnn模型(onmt.modules.StackedLSTM/GRU)列表
- self.context_gate: 指定使用的context_gate的类型，choices = ['source', 'target', 'both']。当使用input_feed时，context_gate作为权值，权衡contxt和原始input之间的比重
- self.dropout: dropput模型。
- self.attn: attention模型，可以指定attention的类型，choices = ['dot', 'general', 'mlp']
- self._copy: 指定是否训练copy attention模型（onmt.modules.GlobalAttention）， action = 'store_true'

#### 方法
- __init__(self, opt, dicts)<br>
- forward(self, input, src, context, state): context [src_len, batch, rnn_size], state是一个用于初始化decode的对象

#### 图
还木画

## NMTModel

#### 属性
- self.multigpu: boolean是否使用multi gpu
- self.encoder
- self.decoder

#### 方法
- __init__(self, encoder, decoder, multigpu = False)
- _fix_enc_hidden(sellf, h): change [layers*directions, batch, dim] to [layers, batch, directions*dim]
- init_decoder_state(self, context, enc_hidden)
- forward(self, src, tgt, lengths, dec_state = None)

### RNNDecoderState(DecoderState)

#### 属性
self.hidden: rnn中的可训练变量[]

dict features不是很了解，有什么features

## onmt.Loss.py
### NMTCriterion
设置loss function为NLLLoss，初始化weight使得忽略对应特殊字符。
#### 参数
- vocabulary Size
- opt
#### 返回
Loss function

### ShardVariable
？？？？不知道是要干什么的？？？

### collectGrads
？？？？搞不懂啊，和上一个方法相关，等遇到了在看？？？

### class Statistics
计算各种统计数据
#### 初始化参数
- loss = 0 
- n_words = 0
- n_correct = 0

#### 属性(都表示字面意思)
- self.loss = loss
- self.n_words = n_words
- self.n_correct = n_correct
- self.n_str_words = 0
- self.start_time = time.time() (import time)

#### 方法
- _init_(self, loss = 0, n_words = 0, n_correct = 0):
- update(self, stat): stat应该是statistics类型的，把stat中的属性，加到目前的属性中
- accuracy(self): 返回$100*(self.n_correct / float(self.n_words)$
- ppl(self): 返回$math.exp(min(self.loss/ self.n_words, 100))$
- elapsed_time(self): 放回当前时间减去刚生成这个实例时的时间
- output(self, epoch, batch, n_batches, start): 统计前面方法中得到的信息，进行输出
- log(self, prefix, experiment, optim): 把信息加到experiment中

### MemoryEfficientLoss
这个先跳了？？？？？应该先看一下什么是copygenerator，否则实在看不懂啊？？？


## onmt.modules.CopyGenerator
###  class CopyGenerator(nn.Module)
暂时不知道为什么需要copyGenerator<br>
Generator module that additionally considers copying words directly from the source
#### 属性
- self.linear: [rnn_size, tgt_dict.size()]，用于计算目标词汇表各个单词的概率
- self.linear_copy: [rnn_size, 1]，用于计算copy的概率
- self.src_dict
- self.tgt_dict: target dict

#### 方法
- __init__(self, opt, src_dict, tgt_dict)
- forward(self, hidden, attn, verbose = False)： args:hidden(FLoatTensor): (tgt_len \* batch) x hidden为什么是tgt_length \* batch，tgt_length是什么？？怀疑他表示目标句子的长度<br>
计算$P(w) = p(z=1)p_{copy}(w|z=0) 和 p(z=0)*P_{softmax}(w|z=0)$，其中p(z=1)表示copy的概率，其值由输入hidden确定。$p_{softmax}$表示目标词汇表中，各个单词出现的概率，其值由输入hidden确定。$p_{copy}$就是输入attn：(tgt_len*length) x src_len
- _debug_copy(self, src, copy, prob, out_prob, attn, mal_attn): 不是很懂这个方法是为了干什么？？？

### CopyCriterion(probs, attn, targ, align, eps = 1e-12): 看不懂啊？？？

## onmt.optim.py
### class Optim(object)
- self.last_ppl: 当前的ppl值
- self.lr: learning rate
- self.max_grad_norm: 设置max_grad_norm，用于clip_grad_norm.
- self.method: 方法
- self.lr_decay: learning rate decay??
- self.start_decay_at
- self.start_decay: boolean
- self._step: 记录目前step的次数，用于辅助计算decay_lr，具体原理不懂？
- self.betas: = [beta1, beta2]用于Adam。coefficients used for computing running averages of gradient and its square (default: (0.9, 0.999))
- self.opt: opt
- self.optimizer: 优化器
#### 方法
- __init__(self, method, lr, max_grad_norm, lr_decay = 1, start_decay_at = None, beta1 = 0.9, beta2 = 9.8, opt = None)
- set_parameters(self, params): 设置optimizer的方法，包括：sgd, adagrad, adadelta, adam
- _setRate(self, lr): 设置optimizer的learning rate
- step(self): 加入了根据是否设置了decay_method, 调整learning rate
- updateLearningRate(self, ppl, epoch): decay learning rate if val perf does not improve or we hit the start_decay_at limit.

## onmt.IO.py
### extractFeatures(tokens)
输入的tokens是一个列表，其中每个项表示单词及其对应的词性，他们通过‘|’分隔开。该方法的功能主要是提取每个token中的word和features。<br>
应该注意的是，每个单词可以有多个features，但是每个单词的features的总数应该是相同的。<br>
该方法返回words, features(列表，len(features)代表feature的个数), numFeatures

### merge_vocabs(vocabs, vocab_size = None)
合并多个词汇表，一般词汇表从特定的文档汇中，归纳得到，不同文档得到的词汇表之间，存在差异。这个方法，主要用来合并这些词汇表。vocab_size指定最后生成的词汇表的大小，None的话就是没有限定。<br>
用try: merged[word] += count except: merged[word] = 0，也是可以的，但不知道优劣在哪里。<br>
返回用到torchtext，不知道是在干什么？？还有vocab也是torchtext里面的，所以只能表示懵逼了？？？

### class OrderedIterator(torchtext.data.Iterator)
父类是什么鬼？？？

### class ONMTDataset(torchtext.data.Dataset):
父类是什么鬼？？？
以torchtext.data.Dataset为基础，打造一个使用与MT任务的Dataset类。
#### 外部方法有
collapseCopyScores(self, scores, batch, tgt_vocab)

## train.py
#### 读取参数并做一些设置

In [ ]:
//提取选项
parser = argparse.ArgumentParser(description='train.py')
opt = parser.parse_args() 

// 设定层数和embedding vector的长度。
// 若没有特殊指定，默认为2和500
opt.src_word_vec_size = opt.tgt_word_vec_size = opt.word_vec_size
opt.enc_layers = opt.dec_layers = opt.layers

// 设定是否用双向encoder
opt.brnn = (opt.encoder_type == 'brnn')

// 设定seed，并不知道这一步，优在哪里？？？
if opt.seed>0: torch.manual_seed(opt.seed)
    
// 若存在n卡，那么最好就用显卡跑吧，关于显卡的一切，目前表示未知？？？
if torch.cuda.is_available() and opt.gpuid: 
    cuda.set_device(opt.gpuid[0]) 
    if opt.seed > 0: torch.cuda.manual_seed(opt.seed)
        
// 设定远端log服务器crayon，若当前experiment已经存在，则覆盖之。具体使用同样表示未知？？？
if opt.exp_host != "":
    from pycrayon import CrayonClient
    cc = CrayonClient(hostname = opt.exp_host)
    if opt.exp in cc.get_experiment_names(): cc.remove_experiment(opt.exp)
    experiment = cc. create_experiment(opt.exp)

#### 实现main方法

In [1]:
# 读取数据到cpu, 设置一些变量。不知道什么样的数据才能使用这种方法读取，只有通过torch.store的数据吗？
# 不是很清楚这些数据是用来干嘛的，估计train存储了训练用的数据，field存储了训练相关的一些信息，像词汇表之类的。
# 但是为什么要一次性的读取到cpu呢，对于这点表示不解，特别是训练数据，不怕太大吗？？？
train = torch.load(opt.data + '.train.pt', pickle_module = dill)
field = torch.load(opt.data + '.fields.pt', pickle_module = dill)
valid = torch.load(opt.data + '.valid.pt', pickle_module = dill)
fields = dict(fields)
src_features = [fields["src_feat_"+ str(j)] for j in range(train.nfeatures)]
checkpoint = None

# 如果checkpoint中有训练信息，则从中读取
dict_checkpoint = opt.train_from
if dict_checkpoint:
    checkpoint = troch.load(dict_checkpoint, map_location = lambda storage, loc: storage) //为什么这么remap
    fields = checkpoint['field']
    
# 从下面两个输出可以看出不少东西：
# fields主要包括src,tgt和各种feature记录, vocab表示对应项的值域，比如src就是源词汇表。
# 一个问题是这些src_feature是从checkpoint中，还是opt.data中得来的呢？？？？？？
print(' * vocabulary size. source = %d; target = %d'%(len(fields['src'].vocab), len(fields['tgt'].vocab)))
for j, feat in enumerate(src_features):
    print(' * src feature %d size = %d' %(j, len(feat.vocab)))
# 证实train确实是训练数据
print('* number of training sentences. %d'%len(train))
print('* maxium batch size. %d' % opt.batch_size)

""" 下面开始建模，主要操作包括
1. 分析是否有src_feature，有的话embedding的时候应该加入考虑
2. 根据opt决定encoder和decoder，并构建NMT模型
3. 决定是否训练copy attention层，这个不是很了解，没见过？？？？？？
4. 是否有checkpoint，有的话就读取参数
5. 是否显卡运行
"""
cuda = (len(opt.gpuid) >= 1)
model = onmt.Models.make_base_model(opt, opt, fields, cuda, checkpoint)

# 如果读取了checkpoint，调整起始epoch, 然而模型的读取并不是在这里进行
if opt.train_from:
    opt.start_epoch = checkpoint['epoch'] + 1

# 如果存在多快显卡，设置并行
if len(opt.gpuid) > 1:
    model = nn.DataParallel(model, device_ids = opt.gupid, dim = 1)

""" 如果没有从checkpoint读取数据，那么就进行初始化
1. 使用uniform distribution进行初始化，如果param_init为了0，则取消初始化，否则使用uniform(-param_init, param_init)
2. 如果提前训练有embedding，则直接读取
3. 设置优化器(如果存在checkpoint，则直接读取)估计是合作编程，这里写得有点乱
"""
if not opt.train_from:
    if opt.param_init != 0.0:
        for p in model.parameter():
            p.data.uniform_(-opt.param_init, opt.param_init)
    model.encoder.embeddings.load_pretrained_vectors(opt.pre_word_vecs_enc)
    model.decoder.embeddings.load_pretrained_vectors(opt.pre_word_vecs_dec)
    optim = onmt.Optim(
        opt.optim, opt.learning_rate, opt_max_grad_norm, 
        lr_decay = opt.learning_rate_decay,
        start_decay_at = opt.start_decay_at,
        opt = opt
    )
else:
    optim = checkpoint['optim']
optim.set_parameters(model.parameters())

# 训练模型
trainModel(model, train, valid, fields, optim)


SyntaxError: invalid syntax (<ipython-input-1-7d9699878920>, line 14)

In [ ]:
#